In [ ]:
#국가교통정보센터ITS: https://its.go.kr/
#영동고속도로 CCTV

import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')


# 영상 스트리밍 URL 또는 로컬 비디오 파일 경로
video_url = "https://cctvsec.ktict.co.kr:8091/3209/8GFonP9foco/eDpo8LA+GeyjiwtTBFeDq1FYihZTkXWTDEL7rXbJgBVqgtghLRVZ"

# VideoCapture 객체 초기화
cap = cv2.VideoCapture(video_url)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


In [6]:
import requests
import cv2
import torch
from xml.etree import ElementTree as ET

# YOLO 모델 불러오기
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  

# API 엔드포인트 URL 및 매개 변수 설정
api_url = "https://openapi.its.go.kr:9443/cctvInfo"
params = {
    "apiKey": "848b2f28c3184279a1935e17289e0009",  # 실제 API 키
    "type": "all",
    "cctvType": "1",
    "minX": "126.800000",
    "maxX": "127.890000",
    "minY": "34.900000",
    "maxY": "35.100000",
    "getType": "xml"
}

# GET 요청 보내기
response = requests.get(api_url, params=params)

# 응답 코드 확인
print("Response code:", response.status_code)

if response.status_code == 200:
    # XML 파싱
    root = ET.fromstring(response.text)

    # CCTV URL 추출
    cctv_data = root.findall(".//data")
    for data in cctv_data:
        cctv_url = data.find("cctvurl").text

        # OpenCV로 비디오 스트리밍 실행
        cap = cv2.VideoCapture(cctv_url)

        while cap.isOpened():
            ret, frame = cap.read()

            if ret:
                # YOLO로 차량 감지
                results = model(frame)
                results.render()  # 결과를 프레임에 렌더링

                cv2.imshow("CCTV Streaming", frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                break

        cap.release()
        cv2.destroyAllWindows()
else:
    print("Failed to fetch CCTV data")


Using cache found in C:\Users\rules/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-11-3 Python-3.10.0 torch-2.1.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Response code: 200
